In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

In [2]:
# Hyper-parameters 
input_size = 784
num_classes = 10
num_epochs = 5
batch_size = 100
learning_rate = 0.001

In [3]:
# MNIST dataset (images and labels)
train_dataset = torchvision.datasets.MNIST(root='./data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

Processing...
Done!


In [4]:
# Data loader (input pipeline)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

In [5]:
# Logistic regression model
model = nn.Linear(input_size, num_classes)

# Loss and optimizer
# nn.CrossEntropyLoss() computes softmax internally
criterion = nn.CrossEntropyLoss()  
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

In [6]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Reshape images to (batch_size, input_size)
        images = images.reshape(-1, 28*28)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))


Epoch [1/5], Step [100/600], Loss: 2.2045
Epoch [1/5], Step [200/600], Loss: 2.1330
Epoch [1/5], Step [300/600], Loss: 2.0046
Epoch [1/5], Step [400/600], Loss: 1.8857
Epoch [1/5], Step [500/600], Loss: 1.8108
Epoch [1/5], Step [600/600], Loss: 1.7921
Epoch [2/5], Step [100/600], Loss: 1.7033
Epoch [2/5], Step [200/600], Loss: 1.7629
Epoch [2/5], Step [300/600], Loss: 1.6339
Epoch [2/5], Step [400/600], Loss: 1.5506
Epoch [2/5], Step [500/600], Loss: 1.4557
Epoch [2/5], Step [600/600], Loss: 1.5270
Epoch [3/5], Step [100/600], Loss: 1.4109
Epoch [3/5], Step [200/600], Loss: 1.4016
Epoch [3/5], Step [300/600], Loss: 1.3531
Epoch [3/5], Step [400/600], Loss: 1.3038
Epoch [3/5], Step [500/600], Loss: 1.4321
Epoch [3/5], Step [600/600], Loss: 1.2274
Epoch [4/5], Step [100/600], Loss: 1.1536
Epoch [4/5], Step [200/600], Loss: 1.2410
Epoch [4/5], Step [300/600], Loss: 1.0501
Epoch [4/5], Step [400/600], Loss: 1.2284
Epoch [4/5], Step [500/600], Loss: 1.0421
Epoch [4/5], Step [600/600], Loss:

In [7]:
# Test the model
# In test phase, we don't need to compute gradients (for memory efficiency)
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum()

    print('Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the model on the 10000 test images: 82 %


In [8]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')